In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
df = pd.read_csv("10%_Non_Bankrupty.csv", sep=',', header=0, encoding='cp950')

In [ ]:
num_rows = df.shape[0]
print("Total number of rows in the DataFrame:", num_rows)

In [ ]:
print(df.head(5))

In [ ]:
df['date'] = pd.to_datetime(df['public_date'])
df = df.sort_values(by=['permno', 'public_date'])

# Ratios based on Feature Selection using LASSO
ratios = ['X2', 'X3', 'X5', 'X6', 'X7', 'X8', 'X9', 'X12', 'X13', 'X14', 'X15', 'X16', 
          'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 
          'X28', 'X29', 'X31', 'X32', 'X33', 'X34', 'X35', 'X37', 'X38', 'X40', 'X41', 
          'X46', 'X48']

In [ ]:
# 1. Calculate the Mean and Standard Deviation for Each Financial Ratio
means = df[ratios].mean()
stds = df[ratios].std()

In [ ]:
# 2. Apply the Brightness Transformation
def transform_brightness(value, mean, std):
    if pd.isna(value) or std == 0:
        return 128
    brightness = (value - mean) / std * 100 + 128
    return min(max(brightness, 0), 255)

for ratio in ratios:
    df[ratio] = df.apply(lambda row: transform_brightness(row[ratio], means[ratio], stds[ratio]), axis=1)

In [ ]:
# 3. Create the Visualization
unique_companies = df['permno'].unique()
for company in unique_companies:
    company_data = df[df['permno'] == company]

    # Sort in descending order by date
    company_data = company_data.sort_values(by='date', ascending=False)
    # Skip companies with fewer than 36 months of data
    if len(company_data) < 36:
        continue  
    # Select the last (or only) 36 months of data
    company_data = company_data.head(36)

    # Sort again in ascending order so the data is from oldest to newest
    company_data = company_data.sort_values(by='date', ascending=True)

    # Adjust missing data here using ffill(), bfill(), interpolate()
    company_data[ratios] = company_data[ratios].interpolate().ffill().bfill()

    img_data = company_data[ratios].values

    plt.figure(figsize=(8, 8))  # Set a specific size for plots
    cax = plt.imshow(img_data, aspect='auto', cmap='gray', vmin=0, vmax=255)
    plt.xticks([])
    plt.yticks([])
    # Saving the image
    folder_path = 'Data/Non_Bankruptcy' 
    file_path = os.path.join(folder_path, f'company_{company}_image.png')
    plt.savefig(file_path, bbox_inches='tight', pad_inches=0)
    plt.close('all')
